In [1]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('clustering_in_real_life').getOrCreate()

22/05/26 09:53:35 WARN Utils: Your hostname, ls-Aspire-A515-51G resolves to a loopback address: 127.0.1.1; using 192.168.15.8 instead (on interface wlp3s0)
22/05/26 09:53:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 09:53:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/26 09:53:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/26 09:53:37 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
dataset = spark.read.csv('seeds_dataset.csv', header=True, inferSchema=True)

In [3]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [4]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [6]:
# Use your domain knowledge
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [7]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [8]:
assembler =  VectorAssembler(inputCols=dataset.columns, 
                            outputCol='features')

In [9]:
final_data = assembler.transform(dataset)

In [10]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
from pyspark.ml.feature import StandardScaler

In [12]:
scaler = StandardScaler(inputCol='features', 
                       outputCol='scaledFeatures')

In [13]:
scaler_model = scaler.fit(final_data) # Collect std information

In [14]:
final_data = scaler_model.transform(final_data) # Transform the data

In [18]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [19]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [20]:
model = kmeans.fit(final_data)

In [23]:
model.distanceMeasure

Param(parent='KMeans_a50b2b61fd14', name='distanceMeasure', doc="the distance measure. Supported options: 'euclidean' and 'cosine'.")

In [24]:
centers = model.clusterCenters()

In [25]:
centers

[array([ 4.91309043, 10.92012526, 37.32658724, 12.37724251,  8.59393872,
         1.8226071 , 10.35389957]),
 array([ 6.32636687, 12.38115343, 37.39222755, 13.9206997 ,  9.75485787,
         2.41428142, 12.28078861]),
 array([ 4.0648023 , 10.14242485, 35.82143905, 11.81918014,  7.51855717,
         3.19361875, 10.40520609])]

In [27]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows

